In [127]:
from icecream import ic
from ordered_set import OrderedSet
from itertools import product
import itertools as _itertools
import stim
from idtcorev2 import jacobian_coefficient_calc

# Update important parameters here!
VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV

In [128]:
num_qubits = 3
max_weight = 2
term_dict = {("H", "XIX"): 0.0001}

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [129]:
HS_index_iterator = stim.PauliString.iter_all(
    num_qubits, min_weight=1, max_weight=max_weight
)

pauli_node_attributes = list([p for p in HS_index_iterator])
ca_pauli_node_attributes = list(_itertools.combinations(pauli_node_attributes, 2))

def ca_pauli_weight_filter(pauli_pair, max_weight):
    used_indices_1 = set(
        i for i, ltr in enumerate(str(pauli_pair[0])[1:]) if ltr != "_"
    )
    used_indices_2 = set(
        i for i, ltr in enumerate(str(pauli_pair[1])[1:]) if ltr != "_"
    )
    intersect = used_indices_1.intersection(used_indices_2)
    if len(intersect) > 0 and len(intersect) <= max_weight:
        return True

ca_pauli_node_attributes = [
    ppair
    for ppair in ca_pauli_node_attributes
    if ca_pauli_weight_filter(ppair, max_weight)
]

measure_string_iterator = stim.PauliString.iter_all(
    num_qubits, min_weight=num_qubits
)
measure_string_attributes = list([p for p in measure_string_iterator])

hs_error_gen_classes = "hs"
ca_error_gen_classes = "c"


hs_experiment = list(
    product(
        hs_error_gen_classes,
        pauli_node_attributes,
        measure_string_attributes,
        measure_string_attributes,
    )
)
ca_experiment = list(
    product(
        ca_error_gen_classes,
        ca_pauli_node_attributes,
        measure_string_attributes,
        measure_string_attributes,
    )
)



import pandas as pd

# df = pd.DataFrame()
jacobian_coef_dict = {"index": OrderedSet(), "columns": OrderedSet()}
data = {}

# These come back as class, index, prep_str, meas_str, observ_str: coef
# I THINK this is right, should double check and write unit tests
for key in hs_experiment + ca_experiment:
    elt = jacobian_coefficient_calc(*key)
    for el in elt:
        if el:
            observable = ",".join(str(s)[1:] for s in el[-4:-1])
            egen = ",".join(str(s) for s in el[:-4])
            coef = int(el[-1])
            jacobian_coef_dict["index"].add(observable)
            jacobian_coef_dict["columns"].add(egen)
            if data.get(egen):
                data[egen].append(coef)
            else:
                data[egen] = [coef]

df = pd.DataFrame(data, index= jacobian_coef_dict["index"], columns=jacobian_coef_dict["columns"])
ic(df)

KeyboardInterrupt: 

In [ ]:
import numpy as np
np.set_printoptions(precision=1, linewidth=1000)

In [ ]:
import pygsti
from pygsti.extras import idletomography as idt
gates = ["Gi", "Gx", "Gy", "Gcnot"]
max_lengths = [1, 2]
pspec = pygsti.processors.QubitProcessorSpec(
        num_qubits, gates, geometry="line", nonstd_gate_unitaries={(): num_qubits, "Gi": np.eye((2**num_qubits))},
        availability={"Gi": [tuple(i for i in range(num_qubits))]},
    )
mdl_target = pygsti.models.create_crosstalk_free_model(pspec)
paulidicts = idt.determine_paulidicts(mdl_target)
idle_experiments = idt.make_idle_tomography_list(
        num_qubits, max_lengths, paulidicts, maxweight=1
    )

In [ ]:
mdl_datagen = pygsti.models.create_crosstalk_free_model(
pspec, lindblad_error_coeffs={"Gi": term_dict},lindblad_parameterization="GLND")
updated_ckt_list = []
from pygsti.baseobjs import Label
for ckt in idle_experiments:
    new_ckt = ckt.copy(editable=True)
    for i, lbl in enumerate(ckt):
        if lbl == Label(()):
            # new_ckt[i] = [Label(("Gi", i)) for i in range(n_qubits)]
            new_ckt[i] = Label("Gi", tuple(i for i in range(num_qubits)))
            # new_ckt[i] = Label(("Gi", 0))
    updated_ckt_list.append(new_ckt)

# Error models! Random with right CP constraints from Taxonomy paper
ds = pygsti.data.simulate_data(
    mdl_datagen, updated_ckt_list, 10000000, seed=8675309, sample_error="none"
)
ds

C:\Users\jkskolf\pyGSTi\pygsti\circuits\circuit.py:816: UserWarning: Editable circuit is being converted to read-only mode in order to hash it.  You should call circuit.done_editing() beforehand.
  _warnings.warn(("Editable circuit is being converted to read-only"


In [ ]:
from idttools import allerrors, all_full_length_observables, alloutcomes
import collections as _collections
from pygsti.circuits.circuit import Circuit as _Circuit
def report_observed_rates(nqubits,
    dataset,
    max_lengths,
    pauli_basis_dicts,
    maxweight=2,
    idle_string=((),),):
    all_fidpairs = dict(enumerate(idt.idle_tomography_fidpairs(nqubits)))
    # print(all_fidpairs)
    if nqubits == 1:  # special case where line-labels may be ('*',)
        if len(dataset) > 0:
            first_circuit = list(dataset.keys())[0]
            line_labels = first_circuit.line_labels
        else:
            line_labels = (0,)
        GiStr = _Circuit(idle_string, line_labels=line_labels)
    else:
        GiStr = _Circuit(idle_string, num_lines=nqubits)
    obs_infos = dict()
    errors = allerrors(nqubits, maxweight)
    fit_order = 1
    observed_error_rates = {}
    obs_error_rates_by_exp = {}
    whatever = {}
    for ifp, pauli_fidpair in all_fidpairs.items():
        all_observables = all_full_length_observables(
            pauli_fidpair[1], nqubits
        )
        all_outcomes = idt.idttools.allobservables(pauli_fidpair[1], maxweight)
        infos_for_this_fidpair = _collections.OrderedDict()
        for j, out in enumerate(all_outcomes):
            info = idt.compute_observed_err_rate(
                dataset,
                pauli_fidpair,
                pauli_basis_dicts,
                GiStr,
                out,
                max_lengths,
                fit_order,
            )

            #info["jacobian row"] = full_jacobian[ifp]
            infos_for_this_fidpair[out] = info
            # ic(infos_for_this_fidpair)
            
            obs_infos[ifp] = infos_for_this_fidpair
            observed_error_rates[ifp] = [
                info["rate"] for info in infos_for_this_fidpair.values()
            ]
            obs_error_rates_by_exp[str(pauli_fidpair[0]).replace("+",""), str(pauli_fidpair[1]).replace("+",""), str(out).replace("+","").replace("I","_")] = [
                info["rate"] for info in infos_for_this_fidpair.values()
            ][-1]
            # obs_err_rates = np.concatenate([np.array([
            #                 observed_error_rates[i]
            #                 for i in range(len(all_fidpairs))
            #                 ]
            #             )
            #         ]
            #     )
        whatever[pauli_fidpair] = 1
    return observed_error_rates, obs_error_rates_by_exp

In [ ]:
idl_string = "Gi:"+":".join((str(i) for i in range(num_qubits)))
observed_error_rates, obs_error_rates_by_exp = report_observed_rates(num_qubits, ds, max_lengths, paulidicts, idle_string=idl_string)

In [ ]:
observed_error_rates

{0: [-5.1730544255886464e-17],
 1: [0.0],
 2: [0.0],
 3: [0.0],
 4: [-0.00019994002864204022],
 5: [0.0],
 6: [0.0],
 7: [0.0],
 8: [-0.00019994002864204022]}

In [ ]:
obs_error_rates_by_exp

{('X', 'X', ' X'): -5.1730544255886464e-17,
 ('X', 'Y', ' Y'): 0.0,
 ('X', 'Z', ' Z'): 0.0,
 ('Y', 'X', ' X'): 0.0,
 ('Y', 'Y', ' Y'): -0.00019994002864204022,
 ('Y', 'Z', ' Z'): 0.0,
 ('Z', 'X', ' X'): 0.0,
 ('Z', 'Y', ' Y'): 0.0,
 ('Z', 'Z', ' Z'): -0.00019994002864204022}

{('X', 'X', ' X'): -5.1730544255886464e-17,
 ('X', 'Y', ' Y'): 0.0,
 ('X', 'Z', ' Z'): 0.0,
 ('Y', 'X', ' X'): 0.0,
 ('Y', 'Y', ' Y'): -0.00019994002864204022,
 ('Y', 'Z', ' Z'): 0.0,
 ('Z', 'X', ' X'): 0.0,
 ('Z', 'Y', ' Y'): 0.0,
 ('Z', 'Z', ' Z'): -0.00019994002864204022}

In [ ]:
obs_rats = [v for v in obs_error_rates_by_exp.values()]

In [ ]:
j = df.to_numpy()

sanity check: jacobian constructed is full rank

In [ ]:
jinv = np.linalg.pinv(j)

In [ ]:
intrins_errs = jinv @ obs_rats

# Supposed intrinsic error rates (wrong)

In [ ]:
dict(zip(df.columns,intrins_errs))

{'H,+X': -1.3448205832742842e-20,
 'H,+Y': -2.2215558737998726e-05,
 'H,+Z': 2.221555873799872e-05,
 'S,+X': 6.479537965250704e-05,
 'S,+Y': 9.256482807510254e-06,
 'S,+Z': 9.256482807510238e-06,
 'C,+X,+Y': 3.7025931230041086e-06,
 'C,+X,+Z': 3.7025931230041014e-06,
 'C,+Y,+Z': 2.591815186100283e-05,
 'A,+X,+Y': 1.2959075930501393e-05,
 'A,+X,+Z': -1.2959075930501391e-05,
 'A,+Y,+Z': -9.256482807497321e-06}